In [12]:
def set_machines(machines, typee):
    setmachines = [];
    for i in machines:
        if i.type == typee:
            setmachines.append(i)
    return setmachines;
def min_time_machines(machines, typee = None):
    if typee is not None:
        machines = set_machines(machines, typee)
    mintime = machines[0].time_taken;
    for i in machines:
        if i.time_taken < mintime:
            mintime = i.time_taken;
    return mintime;
def max_time_machines(machines, typee = None):
    if typee is not None:
        machines = set_machines(machines, typee)
    maxtime = machines[0].time_taken;
    for i in machines:
        if i.time_taken > maxtime:
            maxtime = i.time_taken;
    return maxtime;

In [32]:
def normalize_values(rows, columns):
    if len(rows)>0:
        min_max = [0]*len(columns);
        for i in range(len(columns)):
            min_max[i] = [rows[0][columns[i]],rows[0][columns[i]]];
        for row in rows:
            for j in range(len(columns)):
                min_max[j][0] = min(row[columns[j]], min_max[j][0]);
                min_max[j][1] = max(row[columns[j]], min_max[j][1]);
        for row in rows:
            for j in range(len(columns)):
                range_mm = min_max[j][1] - min_max[j][0];
                if range_mm > 0:
                    row[columns[j]] = (row[columns[j]]-min_max[j][0])/range_mm;
                else:
                    row[columns[j]]=0;
def test_normalize_values():
    #NOTA: recuerda que se normaliza hacia "abajo"
    test_arr = [[1,1,1,1,2],[2,2,2,1,3],[2,3,2,1,2],[3,4,3,1,1]]
    normalize_values(test_arr, [0,1,3,4])
    print(test_arr)

In [14]:
class JSSP_data_gatherer():
    def __init__(self):
        pass
    def get_curr_state(self, activities, machines):#, data):    
        all_times = 0;
        pending_times = 0;
        n_pending = 0;
        pnd_times = [];
        for j in activities:
            for i in j.activities:
                all_times += i.time_cost;
            pnd_times.extend(j.pendant_activities());


        for i in pnd_times:
            pending_times +=i;

        n_pending = len(pnd_times);
        pending_sd = 0;

        if n_pending <= 0:
            pending_avg = 0;
        elif n_pending == 1:
            pending_avg = pending_times;
        else:
            pending_avg = pending_times / n_pending;
            for i in pnd_times:
                pending_sd += (i-pending_avg)*(i-pending_avg);
            pending_sd = math.sqrt(pending_sd/(n_pending-1));

        processed_times = 0;
        n_processed = 0;
        pro_times = [];
        for i in range(len(machines)):
            #per_machine[i] = [];
            for j in (machines[i].machine):
                processed_times +=j.time_cost;
                n_processed +=1;
                pro_times.append(j.time_cost)

        processed_sd = 0;
        if n_processed <= 0:
            processed_avg = 0;
        elif n_processed == 1:
            processed_avg = processed_times;
        else:
            processed_avg = processed_times / n_processed;
            for i in pro_times:
                processed_sd += (i-processed_avg)*(i-processed_avg);
            processed_sd = math.sqrt(processed_sd/(n_processed-1));
        max_machines = machines[0].time_taken;
        unused_time_machine = 0;
        for i in machines:
            unused_time_machine += i.unused_time();
            if i.time_taken> max_machines:
                max_machines = i.time_taken;

        def divide(a, b):
            if b == 0:
                return 0;
            return a/b;



        ATP = divide(processed_times, all_times);
        DPT = divide(processed_sd, processed_avg);
        SLACK = divide(unused_time_machine,(max_machines*len(machines)));
        DNPT = divide(pending_sd, pending_avg);
        NATP = divide(pending_times, all_times);
        NJT = divide(pending_times, n_pending);
        gen_info =  [ATP, DPT, SLACK, DNPT, NATP, NJT];
        return gen_info
    
    def print_state(self, activities, machines, step = None):
        #Object activity
        choosen_activity = None
        if step is not None:
            choosen_activity = activities[step["activity"][0]].activities[step["activity"][1]]
        final_table = [];
        target = []
        state_activities = []
        #Get next available activity from each job
        for j in activities:
            act = j.next_free_activity()
            if act is not None:
                #earliest time a machine can be place by its own restrictions
                et = act.earliest_time();
                #min time of a possible machine (confusing if used or not as yo can place it in any position of the machine, but then the min time would be the 'et')
                emt= min_time_machines(machines, act.machine_type); 
                emmt = max_time_machines(machines, act.machine_type)
                if choosen_activity is not None and act == choosen_activity:
                    target.append(True)
                else:
                    target.append(False)
                state_activities.append(act)
                
                #self.data_acts['feature_names'] = ['job_id', 'act_id', 'cost', 'cost_total', 'time left job','earliest time activity', 'earliest possible time','wasted time', 'least loaded machine time', 'most loaded machine time', 'final time']
                final_table.append([act.activities.id, act.id, act.time_cost, act.normalized_time, act.activities.remaining_time, et, max(et, emt), emt-et, emt, emmt, max(et, emt)+act.time_cost]);
        
        #normalize values for pendant_job_id', 'earliest time', 'earliest possible time','wasted time'
        #these are normalized so 0 will mean the same as least and 1 as most 
        normalize_values(final_table, [0,1,2,4,5,6,7,8,9,10])
        
        gen_info = get_curr_state(activities, machines)
        #As this info is the same for everyone place it on every row
        for i in final_table:
            i.extend(gen_info)
        return final_table, target, state_activities
    
    def get_post_steps_data(self, activities, machines, next_step = None):
        steps = self.getNextPossibleSteps(activities, machines, next_step)
        step_data = []
        step_labels = []
        step_activities = []
        #for adding the other features, as it is only activity ids etc, you can get that data easely, also et had its function, just send that data
        # you could try to find deatures that consider the actual time where the act was placed vs its et but thats just an idea
        for i in steps:
            act = activities[i[0]["activity"][0]].activities[i[0]["activity"][1]]
            #earliest time a machine can be place by its own restrictions
            et = act.earliest_time();
            #min time of a possible machine (confusing if used or not as yo can place it in any position of the machine, but then the min time would be the 'et')
            emt= min_time_machines(machines, act.machine_type);
            emmt = max_time_machines(machines, act.machine_type)
            #siempre se pondra al final de la machine
            if self.do_step(i[0], activities, machines):
                #self.data_state['feature_names'] = ['job_id', 'act_id', 'cost', 'cost_total', 'time left job','earliest time activity', 'earliest possible time','wasted time', 'least loaded machine time', 'most loaded machine time', 'final time']
                last_values = [act.activities.id, act.id, act.time_cost, act.normalized_time, act.activities.remaining_time, et, max(et, emt), emt-et, emt, emmt, max(et, emt)+act.time_cost]
                step_data.append(last_values + get_curr_state(activities, machines))
                step_labels.append(i[2])
                step_activities.append(act)
                #return to normal position

                self.do_step(i[1], activities, machines)
            
        normalize_values(step_data, [0,1,2,4,5,6,7,8,9,10])
            
        return step_data, step_labels, step_activities
    
    def get_machine_data(self, activities, machines, act, pre_data = False, post_data = False, next_step = None):

        if not (pre_data or post_data):
            return [], [], [], []


        movements = self.getPossibleMovements(act, machines, next_step)
        #you could check what would happen if an activity changed position in its same machine, but supposedly it is not necessary
        #cuantas actividades movio (lit depende de la posicion en que se puso jaja), ero pues movera las que iban despues de el tambien que no deberia de haber
        #tiempo en que se puso y tiempo final (igual y hay que ver si se pueden normalizar por steps los resultados de estas features)

        #earliest time a machine can be place by its own restrictions
        et = act.earliest_time();
        #min time of a possible machine (confusing if used or not as yo can place it in any position of the machine, but then the min time would be the 'et')
        emt= min_time_machines(machines, act.machine_type);

        pre_machine_data = []
        machine_data = []
        machine_labels= []
        steps = []

        for i in movements:
            #i[0] has the movement, get the machine and positio in which it will be inserted
            mach = machines[i[0]["machine"]]
            time_mac = 0
            #dependant_acts = act.dependant_activities();
            if i[0]['position']!=0:
                #for j in range(i[0]["position"]-1, len(mach.machine)):
                 #   if mach.machine[j] != act:
                  #      dependant_acts += mach.machine[j].dependant_activities()+1;
                if i[0]['position']==1:
                    time_mac = 0
                else:
                    time_mac = mach.machine[i[0]['position']-2].end_time;
            else:
                time_mac = 0
                if len(mach.machine) > 0:
                    time_mac = mach.machine[-1].end_time;

            #Wasted time and better option would be et-last time of machine that goes before and just change postion and max(0, eso)
            wastedfrommachine = max(0, et-time_mac)
            wastedfromact = max(0, time_mac-et)  

            #max size es sacar la max time machines d la maquinas
            #dependantactivitieses checar si la siguiente activity tiene pos -1 (haz una funcion interna)
            #i2 is the label
            moves = i[2]
            if i[0]['position']<=0:
                moves = 0
            else:
                moves -= (i[0]['position']-1)


            machine_labels.append(i[3])
            steps.append(i[0])
            
            
            
            #['will_move','waste_machine','waste_act', 'cost_total', 'time taken machine', 'position weight', 'new_makespan']
            if pre_data:
                curr_state_values = self.get_curr_state(jssp.activities, jssp.machines)
                pre_machine_data.append([moves, wastedfrommachine, wastedfromact, act.normalized_time, mach.time_taken, time_mac]+curr_state_values)
            if post_data and self.do_step(i[0], activities, machines):

                max_time = max_time_machines(machines)


                #poner tiempo en el que se podia poner - tiempo final
                tf = act.starting_time
                max_time = max_time_machines(machines)

                last_values = [moves, wastedfrommachine, wastedfromact, act.normalized_time, mach.time_taken, time_mac, max_time] 

                machine_data.append(last_values + self.get_curr_state(activities, machines))

                #return to normal position
                self.do_step(i[1], activities, machines)

        if post_data:
            normalize_values(machine_data, [0,1])
        if pre_data:
            normalize_values(pre_machine_data, [0])

        return pre_machine_data, machine_data, machine_labels, steps


In [15]:
class JSSP_step_obtainer():
    #Get steps for every next activity from each job and give the step to place it at the end of the first available machine
    def getNextPossibleSteps(self, activities, machines, next_step = None):
        steps = []        
        for i in activities:
            tempActivity = i.next_free_activity();            
            if tempActivity != None:                
                e_machines = set_machines(machines, tempActivity.machine_type)
                #print(tempActivity.machine_type, e_machines)
                if next_step is not None and next_step["machine"] == e_machines[0].id and next_step["activity"][0] == tempActivity.activities.id and next_step["activity"][1]== tempActivity.id: #and next_step["position"] == -1 or last
                    steps.append([{"machine": e_machines[0].id, "activity": [tempActivity.activities.id, tempActivity.id], "position": 0}, {"machine": tempActivity.machine, "activity": [tempActivity.activities.id, tempActivity.id], "position": tempActivity.position}, True])                        
                else:
                    steps.append([{"machine": e_machines[0].id, "activity": [tempActivity.activities.id, tempActivity.id], "position": 0}, {"machine": tempActivity.machine, "activity": [tempActivity.activities.id, tempActivity.id], "position": tempActivity.position}, False])
        return steps                    
    
    #Get all possible positions of an activity in a machine
    def getPossibleMovements(self, tempActivity, machines, next_step = None):
        movements = []        
        #from possible machines get from 0 to #elements in machine
        possible_machines = set_machines(machines, tempActivity.machine_type)
        
        for i in possible_machines:
            if next_step is not None and next_step["machine"] == i.id and (next_step["position"]<=0 or next_step["position"]>t):
                movements.append([{"machine": i.id, "activity": next_step["activity"], "position": 0}, {"machine": tempActivity.machine, "activity": [tempActivity.activities.id, tempActivity.id], "position": tempActivity.position},t, True])
            else:
                movements.append([{"machine": i.id, "activity": next_step["activity"], "position": 0}, {"machine": tempActivity.machine, "activity": [tempActivity.activities.id, tempActivity.id], "position": tempActivity.position}, t, False])

            t = len(i.machine)
            #check for other possible positions
            start = i.check_min_position(tempActivity)
            for j in range(start,t+1):
                if next_step is not None and next_step["machine"] == i.id and (next_step["position"] == j or (j==0 and (next_step["position"]<=0 or next_step["position"]>t))):
                    movements.append([{"machine": i.id, "activity": next_step["activity"], "position": j}, {"machine": tempActivity.machine, "activity": [tempActivity.activities.id, tempActivity.id], "position": tempActivity.position},t, True])
                else:
                    movements.append([{"machine": i.id, "activity": next_step["activity"], "position": j}, {"machine": tempActivity.machine, "activity": [tempActivity.activities.id, tempActivity.id], "position": tempActivity.position}, t, False])
            
        return movements

In [16]:
class solver(JSSP_data_gatherer, JSSP_step_obtainer):
    
    #Initialize with the data that will be collected
    def __init__(self, id, get_pre_data = True, get_post_data = True, get_machine_data = True):
        self.id = id;
        self.steps = 0;

        self.get_pre_data = get_pre_data
        self.get_post_data = get_post_data
        self.get_machine_data = get_machine_data

    #Dummy function for getting a next steo
    def getNextStep(self, activities, machines):
        step = {"machine": -1, "activity": -1, "position": -1};
        return -1
    
    #Funtion to apply the steps
    def do_step(self, step, activities, machines):
        if step["machine"]>=0:
            return machines[step["machine"]].addActivity(activities[step["activity"][0]].activities[step["activity"][1]], step["position"])
        else:
            return activities[step["activity"][0]].activities[step["activity"][1]].remove_from_machines()             
    
    #Funtion to run get Next Step until -1 is given
    def start(self, jssp):
        self.steps = 0;
        while (True):
            
            #GET STEP
            next_step = self.getNextStep(jssp.activities, jssp.machines);            
            
            #CHECK SPECIAL CASES
            if(next_step == -1):
                break;
            if next_step == 'p':
                print(jssp.data_acts)
                print(jssp.data_state)
                print(jssp.data_machines)
                print(jssp.data_pre_machines)
                continue;            
            if next_step["machine"] == -1:
                self.do_step(next_step, jssp.activities, jssp.machines)
                continue
                        
            #GET DATA:
            if self.get_pre_data:
                curr_states_data, curr_states_labels, curr_states_acts = self.print_state(jssp.activities, jssp.machines, next_step)
                      
            if self.get_post_data:
                step_data, step_labels, step_activities = self.get_post_steps_data(jssp.activities, jssp.machines, next_step)
            
            act = jssp.activities[next_step["activity"][0]].activities[next_step["activity"][1]]
            
            if self.get_machine_data:                
                pre_machine_data, machine_data, machine_labels, machine_steps = self.get_machine_data(jssp.activities, jssp.machines, act, self.get_pre_data, self.get_post_data, next_step)          
            
            #APPLY STEP
            if self.do_step(next_step, jssp.activities, jssp.machines):
                
                #SAVE DATA
                if self.get_pre_data:
                    jssp.data_acts['data'].extend(curr_states_data);
                    jssp.data_acts['target'].extend(curr_states_labels);
                    if self.get_machine_data:
                        jssp.data_pre_machines['data'].extend(pre_machine_data);
                        jssp.data_pre_machines['target'].extend(machine_labels);                
                if self.get_post_data:  
                    jssp.data_state['data'].extend(step_data);
                    jssp.data_state['target'].extend(step_labels);
                    if self.get_machine_data:
                        jssp.data_machines['data'].extend(machine_data);
                        jssp.data_machines['target'].extend(machine_labels);
                
                
                #REGISTER MOVEMENT
                act = jssp.activities[next_step["activity"][0]].activities[next_step["activity"][1]]
                act.last_step = self.steps
                self.steps+=1

In [17]:
class manual_solver(solver):
    def getNextStep(self, activities, machines):
        step = {"machine": -1, "activity": -1, "position": -1};
        ins = input("place an 'e' to end or 'p' to print or anything else to continue:  ")
        if ins == 'e':
            return -1
        elif ins =='p':
            return 'p'
        step["activity"] = []
        step["activity"].append(int(input("id job: ")))
        step["activity"].append(int(input("id activity: ")))
        step["machine"] = int(input("id machine: "));
        step["position"] = int(input("position: "));
        self.get_pre_data = True
        self.get_post_data = True
        self.get_machine_data = True
        return step
    

In [18]:
class double_heuristic_solver(solver):
    def __init__(self, id, activity_heuristic, machine_heuristic,collect_data = True):
        self.id = id;
        self.activity_heuristic = activity_heuristic;
        self.machine_heuristic = machine_heuristic;
        self.steps = 0;
        self.get_pre_data = collect_data
        self.get_post_data = collect_data
        self.get_machine_data = False
        
    def getNextStep(self, activities, machines):
        step = {"machine": -1, "activity": -1, "position": 0};
        step["activity"] = self.activity_heuristic(activities, machines);
        
        if step["activity"] == -1:
            return -1;
        
        step["machine"] = self.machine_heuristic(activities[step["activity"][0]].activities[step["activity"][1]], machines);
        
        #0 is last position
        step["position"] = 0;
        return step

In [19]:
class heuristic_solver(solver):
    def __init__(self, id, heuristic,collect_data = True):
        self.id = id;
        self.heuristic = heuristic;
        self.steps = 0;
        self.get_pre_data = collect_data
        self.get_post_data = collect_data
        self.get_machine_data = False
        
    def getNextStep(self, activities, machines):
        return self.heuristic(activities, machines)
    

In [20]:
class ML_solver(solver):
    def __init__(self, id, ml_alg, ml_mach = None, pre_act= True, pre_mach = True):
        self.id = id;
        self.ml_alg = ml_alg;
        self.ml_mach = ml_mach
        self.steps = 0;
        
        self.get_pre_data = False
        self.get_post_data = False
        self.get_machine_data = False
        
        self.pre_act = pre_act
        self.pre_mach = pre_mach
        
    def getNextStep(self, activities, machines):
        act = self.getNextActivity(activities, machines)   
        if act == -1:
            return -1
        
        if self.ml_mach is None:
            possible_machines = set_machines(machines, act.machine_type)
            if len(possible_machines) == 0:
                return -1
            return {"machine": possible_machines[0].id, "activity": [act.activities.id, act.id], "position": 0}
        
        #get possible positions in machine
        pre_machine_data, machine_data, machine_labels, machine_steps = self.get_machine_data(activities, machines, act, self.pre_mach, not self.pre_mach)
         
        if (self.pre_mach and len(pre_machine_data) <= 0) or (not self.pre_mach and len(machine_data) <= 0):
            return -1
        
        if self.pre_mach:
            machine_data = pre_machine_data
        return self.getBestAnswer(self.ml_mach, machine_data, machine_steps)
    
    def getBestAnswer(self, ml_alg, data, answers):
        best_answer = (None, -1)
        for i in range(len(data)):
            curr_state_val = ml_alg(data[i])
            if curr_state_val > best_answer[1]:
                best_answer = (answers[i], curr_state_val)                
        #print(best_answer[0])   
        return best_answer[0]
    
    def getNextActivity(self, activities, machines):
        if self.pre_act:
            activity_data, activity_labels, acts = self.print_state(activities, machines)
        else:
            activity_data, activity_labels, acts = self.get_post_steps_data(activities, machines)
        
        if len(activity_data) <= 0:
            return -1           
                
        return self.getBestAnswer(self.ml_alg, activity_data, acts)
    

In [21]:
class comparison_solver(solver):
    def __init__(self, id, main_solver, compare_solver):
        self.id = id
        self.main = main_solver
        self.second = compare_solver
        self.steps = 0
        self.act_total = 0
        self.total_score = 0
        self.total_moves = 0
        self.get_pre_data = False
        self.get_post_data = False
        self.get_machine_data = False
        
    def get_accuracy(self):
        if self.total_moves == 0:
            return (0,0)
        
        return (self.act_total * 1.0 / self.total_moves, self.total_score * 1.0 / self.total_moves)
    
    def getNextStep(self, activities, machines):
        real_step = self.main.getNextStep(activities, machines)
        other_step = self.second.getNextStep(activities, machines)
        if real_step == -1:
            return real_step
        
        self.total_moves+=1;
        
        if other_step == -1:
            return real_step
        
        #{"machine": j.id, "activity": [tempActivity.activities.id, tempActivity.id], "position": -1}
        if real_step["activity"][0] == other_step["activity"][0] and real_step["activity"][1] == other_step["activity"][1]:
            self.act_total+=1            
            if real_step["machine"] == other_step["machine"] and real_step["position"] == other_step["position"]:
                self.total_score+=1
        
        return real_step
    
    def change_main(self, new_main):
        self.main = new_main

In [ ]:
class manual_solver(solver):
    def getNextStep(self, activities, machines):
        step = {"machine": -1, "activity": -1, "position": -1};
        ins = 5
        jssp = activities.JSS      
                
                
        while ins>2:        
            ins = input("0 para salir, 1 para nuevo movimiento, 2 para p, 3 para screen clear, 4 pre_acts, 5 post_acts, 6 pre_mach y 7 post_mach\n ")
            if ins == 3:
                clear()
            elif ins ==4:
                print(jssp.data_acts)
            elif ins == 5:
                print(jssp.data_state)
            elif ins == 6:
                print(jssp.data_pre_machines)
            elif ins == 7:
                print(jssp.data_machines)
        
        if ins == 0:
            return -1
        elif ins ==2:
            return 'p'
        step["activity"] = []
        step["activity"].append(int(input("id job: ")))
        step["activity"].append(int(input("id activity: ")))
        step["machine"] = int(input("id machine: "));
        step["position"] = int(input("position: "));
        self.get_pre_data = True
        self.get_post_data = True
        self.get_machine_data = True
        clear()
        return step
    
    def print_curr_JSSP(self, activities, machines):
        #print activities still in their job
        for i in activities:
            next_act = activities.next_id
            next_costs = activities.pendant_activities()
            job = ""
            for j in next_costs:
                job += "["+str(activities.id)+", "+str(next_act)+", "+str(j)+"]"
                next_act+=1            
            print(job)
        print("")
        #print activities in machines
        for i in machines:
            machine = ""
            for j in machines.machine:
                machine += "["+str(j.activities.id)+", "+str(j.id)+", "+str(j.time_cost)+"]"
            print(machine)
    

In [ ]:
#To test the best answer train a tree with the following array (sum>3 is true)
train_arr = [[1,1],[1,2],[2,2],[2,1],[1,0], [0,1],[0,2], [2,0]]
train_labels = [False, True, True, True, False, False, False, False]
#clf = ....
ml_alg = lambda x: clf.predict([x])

#After training use test data as data and answers
test_data = [[1,1],[0,0],[2,1],[1,-1]]#returns [2,1]
test_data = [[1,1],[0,0],[2,0],[1,2]] #returns [1,2]
test_data = [[1,1],[0,0],[2,1],[1,2]] #returns [2,1]


def getBestAnswer(ml_alg, data, answers):
        best_answer = (None, -1)
        for i in range(len(data)):
            curr_state_val = ml_alg(data[i])
            if curr_state_val > best_answer[1]:
                best_answer = (answers[i], curr_state_val)                
        #print(best_answer[0])   
        return best_answer[0]